In [ ]:
import cv2
import random
import numpy as np

import torch
import torch.nn.functional as F
import torchvision.transforms as transforms

from PIL import Image

def pil_to_tensor(pil_image):
  transform = transforms.ToTensor()
  tensor_image = transform(pil_image)
  return tensor_image

def tensor_to_pil(tensor_image):
    transform = transforms.ToPILImage()
    pil_image = transform(tensor_image)
    return pil_image

def load_images_from_folder(folder):
  images = []
  for filename in os.listdir(folder):
    img_path = os.path.join(folder, filename)
    if os.path.isfile(img_path) and img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
      try:
        img = Image.open(img_path)
        images.append(img)
      except Exception as e:
        print(f"Error loading image {img_path}: {e}")
  return images

def draw_boundary_on_image(image_tensor, mask_tensor, boundary_color=(0, 0, 255), boundary_thickness=10):
    if mask_tensor.dtype != torch.uint8:
        mask_tensor = (mask_tensor * 255).to(torch.uint8)

    if image_tensor.dtype != torch.uint8:
        image_tensor = (image_tensor * 255).to(torch.uint8)

    if len(mask_tensor.shape) == 3:
        mask_tensor = mask_tensor[0, :, :]

    mask_np = mask_tensor.numpy().astype(np.uint8)
    contours, _ = cv2.findContours(mask_np, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    image_np = image_tensor.permute(1, 2, 0).numpy().copy()  # [C, H, W] -> [H, W, C]
    cv2.drawContours(image_np, contours, -1, boundary_color, boundary_thickness)
    output_image_tensor = torch.from_numpy(image_np).permute(2, 0, 1)  # 다시 [C, H, W]로 변경
    
    return output_image_tensor

In [ ]:
!git clone https://github.com/ByeongHyunPak/omni-proj.git

import os
os.chdir('/content/omni-proj/omni-proj')

imgs_folder = '/content/omni-proj/imgs/erps' 
images = load_images_from_folder(imgs_folder)

for img in images:
    img = pil_to_tensor(img)
    print(img.shape)

In [ ]:
import utils
from utils.projection_cpu import gridy2x_erp2per, gridy2x_per2erp, gridy2x_erp2fis, gridy2x_fis2erp

def erp2per(hr_erp_img, THETA, PHI, FOVy=75, FOVx=360, HWy=(512, 512)):
    hr_erp_img = pil_to_tensor(hr_erp_img)
    HWx = hr_erp_img.shape[-2:]

    gridy = utils.make_coord(HWy)
    gridy2x, masky = gridy2x_erp2per(
        gridy, HWy, HWx, THETA, PHI, FOVy, FOVx)
    gridy2x = gridy2x.view(*HWy, 2)

    inp = F.grid_sample(hr_erp_img.unsqueeze(0),
                        gridy2x.unsqueeze(0).flip(-1),
                        mode='bicubic',
                        padding_mode='reflection',
                        align_corners=False).clamp_(0, 1)[0]

    gridx = utils.make_coord(HWx, flatten=False)
    gridx2y, maskx = gridy2x_per2erp(
        gridx, HWx, HWy, THETA, PHI, FOVx, FOVy)
    
    maskx = maskx.view(1, *HWx)
    valid_hr_erp_img = hr_erp_img * maskx
    
    return inp, valid_hr_erp_img, maskx

def per2erp(hr_pers_img, THETA, PHI, FOVy=360, FOVx=75, HWy=(512, 1024)):
    hr_pers_img = pil_to_tensor(hr_pers_img)
    HWx = hr_pers_img.shape[-2:]

    gridy = utils.make_coord(HWy)
    gridy2x, masky = gridy2x_per2erp(
        gridy, HWy, HWx, THETA, PHI, FOVy, FOVx)
    gridy2x = gridy2x.view(*HWy, 2)
    masky = masky.view(1, *HWy)

    inp = F.grid_sample(hr_pers_img.unsqueeze(0),
                        gridy2x.unsqueeze(0).flip(-1),
                        mode='bicubic',
                        padding_mode='reflection',
                        align_corners=False).clamp_(0, 1)[0]
    inp = inp * masky

    gridx = utils.make_coord(HWx, flatten=False)
    gridx2y, maskx = gridy2x_erp2per(
        gridx, HWx, HWy, THETA, PHI, FOVx, FOVy)
    
    maskx = maskx.view(1, *HWx)
    valid_hr_pers_img = hr_pers_img * maskx
    
    return inp, valid_hr_pers_img, maskx


def erp2fis(hr_erp_img, THETA, PHI, FOVy=180, FOVx=360, HWy=(1024, 1024)):
    hr_erp_img = pil_to_tensor(hr_erp_img)
    HWx = hr_erp_img.shape[-2:]

    gridy = utils.make_coord(HWy)
    gridy2x, masky = gridy2x_erp2fis(
        gridy, HWy, HWx, THETA, PHI, FOVy, FOVx)
    gridy2x = gridy2x.view(*HWy, 2)
    masky = masky.view(1, *HWy)

    inp = F.grid_sample(hr_erp_img.unsqueeze(0),
                        gridy2x.unsqueeze(0).flip(-1),
                        mode='bicubic',
                        padding_mode='reflection',
                        align_corners=False).clamp_(0, 1)[0]
    inp = inp * masky
    
    gridx = utils.make_coord(HWx, flatten=False)
    gridx2y, maskx = gridy2x_fis2erp(
        gridx, HWx, HWy, THETA, PHI, FOVx, FOVy)

    maskx = maskx.view(1, *HWx)
    valid_hr_erp_img = hr_erp_img * maskx

    return inp, valid_hr_erp_img, maskx

In [ ]:
hr_erp_img = images[0]
display(hr_erp_img)

# ERP to Perspective
THETA = random.uniform(-135, 135)
PHI = random.uniform(-45, 45)

pers_img, valid_hr_erp_img, _ = erp2per(hr_erp_img, THETA, PHI, FOVy=90, FOVx=360)

display(tensor_to_pil(valid_hr_erp_img))
display(tensor_to_pil(pers_img))

# Perspective to ERP
erp_img, valid_lr_pers_img, _ = per2erp(tensor_to_pil(pers_img), THETA, PHI, FOVy=360, FOVx=90)

display(tensor_to_pil(valid_lr_pers_img))
display(tensor_to_pil(erp_img))

display(tensor_to_pil((pil_to_tensor(hr_erp_img) - erp_img).abs()))
display(tensor_to_pil((pil_to_tensor(hr_erp_img) - valid_hr_erp_img).abs()))

# ERP to Fisheye
THETA = random.uniform(-135, 135)
PHI = 0

fish_img, valid_hr_erp_img, _ = erp2fis(hr_erp_img, THETA, PHI, FOVy=180, FOVx=360)

display(tensor_to_pil(valid_hr_erp_img))
display(tensor_to_pil(fish_img))

In [ ]:
# ERP to Perspective
hr_erp_img = images[3]
lr_pers_hw = (512, 512)

num_rows = 4
num_cols = [3, 6, 6, 3]
phi_centers = [-67.5, -22.5, 22.5, 67.5]
phi_interval = 180 // num_rows

for i, n_cols in enumerate(num_cols):
    PHI = phi_centers[i]
    global_hr_erp_ten = pil_to_tensor(hr_erp_img)
    for j in np.arange(n_cols):
        theta_interval = 360 / n_cols
        THETA = j * theta_interval + theta_interval / 2
        pers_img, valid_hr_erp_img, valid_hr_erp_mask =\
            erp2per(hr_erp_img, THETA, PHI, FOVy=90, FOVx=360, HWy=lr_pers_hw)

        hr_erp_ten = pil_to_tensor(hr_erp_img)
        hr_erp_ten = draw_boundary_on_image(hr_erp_ten, valid_hr_erp_mask)

        random_color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        global_hr_erp_ten = draw_boundary_on_image(global_hr_erp_ten, valid_hr_erp_mask, boundary_color=random_color, boundary_thickness=15)

        pers_img = tensor_to_pil(pers_img)
        valid_hr_erp_img = tensor_to_pil(valid_hr_erp_img)
        valid_hr_erp_mask = tensor_to_pil(valid_hr_erp_mask)
        hr_erp_ten = tensor_to_pil(hr_erp_ten)

        display(pers_img)
        display(valid_hr_erp_img)
        display(valid_hr_erp_mask)
        display(hr_erp_ten)
    display(tensor_to_pil(global_hr_erp_ten))
